In [2]:
import pandas as pd
import numpy as np
from fastai.tabular.all import *

In [36]:
df = pd.read_pickle("donors.pkl").iloc[:1000, :]

In [39]:
dateNames = []
for c in df.columns.values:
    dateNames.append(c) if ("DATE" in c) == True else None

In [46]:
# TODO
pd.to_datetime(df[dateNames], infer_datetime_format=True)

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [42]:
pd.to_datetime(df["ODATEDW"])

0     2009-01-01
1     2014-01-01
2     2010-01-01
3     2007-01-01
4     2006-01-01
         ...    
995   2015-01-01
996   2016-01-01
997   2012-01-01
998   2006-01-01
999   2015-01-01
Name: ODATEDW, Length: 1000, dtype: datetime64[ns]

In [7]:
cont_names, cat_names = cont_cat_split(df, max_card=40)

In [13]:
df = df_shrink(df)